In [8]:
import os
import h5py
import numpy as np
np.random.seed(8102)

In [11]:
def slipt(path,ouputs1,ouputs2,nums):
    fid = h5py.File(os.path.expanduser(path),'r')
    labels=np.argmax(fid['label'],1)
    distrib = np.bincount(labels)
    prob = 1/distrib[labels].astype(float)
    prob /= prob.sum()
    ll=len(labels)
    print(ll)
    select=np.random.choice(np.arange(ll), size=nums,replace=False,p=prob)
    labels= np.argmax(np.array([fid['label'][i] for i in select]),1)
    distrib2 = np.bincount(labels)
    print(distrib2)
    print("random seed is "+str(distrib2[-1]==65))
    f=h5py.File(ouputs1)
    t_index=select.tolist()
    t_index.sort()
    label=np.array(fid['label'][t_index])
    sen1=np.array(fid['sen1'][t_index])
    sen2=np.array(fid['sen2'][t_index])
    f.create_dataset('label',data=label)
    f.create_dataset('sen1',data=sen1)
    f.create_dataset('sen2',data=sen2)
    f.close()
    f=h5py.File(ouputs2)
    alltag=np.arange(ll)
    legacy=np.setdiff1d(alltag,select)
    l_index=legacy.tolist()
    l_index.sort()
    label=np.array(fid['label'][l_index])
    sen1=np.array(fid['sen1'][l_index])
    sen2=np.array(fid['sen2'][l_index])
    f.create_dataset('label',data=label)
    f.create_dataset('sen1',data=sen1)
    f.create_dataset('sen2',data=sen2)
    f.close()
    print("done")



In [6]:
slipt('./data/validation.h5','./data/acc1000.h5','./data/mega1.h5',1000)

24119
[66 60 48 52 65 61 67 66 62 58 69 62 49 51 50 49 65]
random seed is True
done


In [12]:
slipt('./data/training.h5','./data/acc3000.h5','./data/mega2.h5',3000)

352366
[192 183 159 151 172 170 189 176 187 185 172 173 167 210 166 174 174]
random seed is False
done


In [9]:
def merge(path1,path2,output):
    f1=h5py.File(os.path.expanduser(path1),'r')
    f2=h5py.File(os.path.expanduser(path2),'r')
    mergef=h5py.File(output)
    label=np.concatenate(
    (np.array(f1['label']),np.array(f2['label'])),axis=0
    )
    sen1=np.concatenate(
    (np.array(f1['sen1']),np.array(f2['sen1'])),axis=0
    )
    sen2=np.concatenate(
    (np.array(f1['sen2']),np.array(f2['sen2'])),axis=0
    )
    mergef.create_dataset('label',data=label)
    mergef.create_dataset('sen1',data=sen1)
    mergef.create_dataset('sen2',data=sen2)
    f1.close()
    f2.close()
    mergef.close()


In [ ]:
merge('./data/mega1.h5','./data/mega2.h5','./data/mega.h5')

In [14]:
merge('./data/acc3000.h5','./data/acc1000.h5','./data/acc4000.h5')

In [23]:
def weighted_data_generate(batch_size,path):
    fid = h5py.File(os.path.expanduser(path),'r')
    labels=np.argmax(fid['label'],1)
    distrib = np.bincount(labels)
    prob = 1/distrib[labels].astype(float)
    prob /= prob.sum()
    lens=len(labels)
    while(True):
        bingo=np.random.choice(np.arange(lens), batch_size, replace=False,p=prob)
        bingo.sort()
        bingo=bingo.tolist()
        y_b = np.array(fid['label'][bingo])
        x_b = np.array(fid['sen2'][bingo])
        yield x_b, y_b
# 使用范例
# f=weighted_data_generate(os.path.expanduser("./data/training.h5"), 512,False)
# for i in range(10):
#     x,y=f.__next__()
#     print(np.sum(y,0))      

In [24]:
f=weighted_data_generate(800,'./data/train372485.h5')
for i in range(10):
    x,y=f.__next__()
    print(np.sum(y,0))

[44. 52. 56. 49. 53. 52. 41. 48. 57. 47. 45. 44. 37. 38. 47. 48. 42.]
[46. 47. 47. 52. 55. 46. 44. 46. 48. 45. 55. 50. 43. 40. 38. 51. 47.]
[57. 49. 50. 41. 46. 43. 46. 39. 50. 49. 49. 49. 48. 48. 45. 47. 44.]
[42. 44. 50. 49. 42. 48. 38. 43. 50. 41. 50. 51. 45. 52. 49. 52. 54.]
[48. 44. 43. 38. 38. 50. 53. 55. 43. 52. 48. 40. 43. 48. 52. 53. 52.]
[46. 46. 51. 37. 39. 33. 51. 51. 45. 57. 46. 66. 34. 48. 52. 45. 53.]
[50. 40. 43. 47. 45. 44. 49. 36. 47. 48. 46. 55. 44. 45. 50. 45. 66.]
[36. 62. 48. 34. 50. 41. 45. 51. 52. 53. 48. 52. 50. 40. 54. 54. 30.]
[50. 43. 47. 51. 44. 52. 47. 58. 47. 42. 45. 66. 44. 39. 38. 32. 55.]
[48. 46. 48. 49. 46. 54. 50. 41. 51. 57. 37. 36. 56. 42. 52. 41. 46.]


In [49]:
a=np.random.choice(np.arange(10), 5, replace=False)
print(a)
b=np.array(range(10))
b[a]

[8 3 2 5 1]


array([8, 3, 2, 5, 1])

In [7]:
fid = h5py.File(os.path.expanduser('./data/mega2.h5'),'r')
labels=np.argmax(fid['label'],1)
distrib = np.bincount(labels)
print(distrib)
a=[1.00048,1.00242,1.00315,1.00085,1.00163,1.00351,1.00030,1.00391,1.00133,1.00117,1.00427,1.00093,1.00089,1.00411,1.00022,1.00077,1.00491]
print(np.argsort(distrib))
print(np.argsort(a))

[ 4876 24248 31534  8500 16321 35120  3080 39150 13397 11769 42730  9341
  8998 41167  2226  7724 49185]
[14  6  0 15  3 12 11  9  8  4  1  2  5  7 13 10 16]
[14  6  0 15  3 12 11  9  8  4  1  2  5  7 13 10 16]
